In [1]:
import plotly.graph_objects as go
from ifsgen.viz.theme import dark_minimal_transparent, dark_minimal_black

In [32]:
def v4d(
    points,
    width=480,
    height=480,
    save=None,
    camera=dict(
        up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=0, y=2.5, z=0)
    ),
):
    a, b, c, d = list(zip(*points))
    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=d,
                y=b,
                z=c,
                mode="markers",
                marker=dict(
                    size=1.5,
                    color=a,  # set color equal to a variable
                    colorscale="Viridis",  # one of plotly colorscales
                    showscale=False,
                    reversescale=True,
                ),
                hoverinfo="skip",
            )
        ]
    )
    fig.update_layout(
        template=dark_minimal_transparent,
        autosize=False,
        width=width,
        height=height,
        margin=dict(l=10, r=10, b=10, t=10, pad=0),
    )
    fig.update_layout(scene_camera=camera)
    return fig
    # if save is not None:
    #     fig.write_html(save)
    # else:
    #     fig.show()

In [4]:
import numpy as np
import random
from typing import Callable

Point = np.ndarray
Edge = np.ndarray
Edges = list[Edge]
PointHistory = list[Point]
EdgeHistory = list[Edge]

In [5]:
def string_to_function(function: str) -> Callable:
    """
    ```python
    equation = "f(A, B) = (A + B)/2"
    function = string_to_function(equation)
    x = function(A=3, B=5)
    assert x == 4.0, "this should pass"
    ```
    """
    declaration, definition = function.split("=")
    declaration = declaration.strip()
    definition = definition.strip()
    assert declaration[0] == "f", "function must start with `f`"
    assert declaration[1] == "(", "function must start with `f(`"
    assert declaration[-1] == ")", "function declaration must end with `)`"
    parameter_string = declaration[2:-1]
    lambda_declaration = "".join(["lambda ", parameter_string, ": "])
    lambda_function = eval(f"{lambda_declaration}{definition}")
    return lambda_function


class BasicIFS:
    def __init__(
        self,
        vertices: Edges,
        midpoint: str | Callable[[Point, Point], Point],
    ) -> None:
        self.selector = lambda e: random.choice(e)
        self.vertices = vertices
        if isinstance(midpoint, str):
            self.midpoint = string_to_function(midpoint)
        else:
            self.midpoint = midpoint

    @property
    def dimensions(self) -> int:
        return len(self.vertices[0])

    def run(
        self,
        iterations: int,
        starting: Point | None = None,
    ) -> list[Point]:
        if starting is None:
            starting = np.zeros(self.dimensions)
        starting_dim = len(starting)
        assert starting_dim == self.dimensions
        point_history = list()
        point_history.append(starting)
        for iter in range(iterations):
            last_point = point_history[iter]
            edge_point = self.selector(self.vertices)
            next_point = self.midpoint(last_point, edge_point)
            point_history.append(next_point)
        return point_history

In [6]:
SIMPLEX = {
    1: [np.array([-1]), np.array([1])],
    2: [
        np.array([-1 / 2, -np.sqrt(3) / 4]),
        np.array([0, np.sqrt(3) / 4]),
        np.array([1 / 2, -np.sqrt(3) / 4]),
    ],
    3: [
        np.array([np.sqrt(8 / 9), 0, -1 / 3]),
        np.array([-np.sqrt(2 / 9), np.sqrt(2 / 3), -1 / 3]),
        np.array([-np.sqrt(2 / 9), -np.sqrt(2 / 3), -1 / 3]),
        np.array([0, 0, 1]),
    ],
    4: [
        np.array([1 / np.sqrt(10), 1 / np.sqrt(6), 1 / np.sqrt(3), 1]),
        np.array([1 / np.sqrt(10), 1 / np.sqrt(6), 1 / np.sqrt(3), -1]),
        np.array([1 / np.sqrt(10), 1 / np.sqrt(6), -2 / np.sqrt(3), 0]),
        np.array([1 / np.sqrt(10), -np.sqrt(3 / 2), 0, 0]),
        np.array([-2 * np.sqrt(2 / 5), 0, 0, 0]),
    ],
}

In [7]:
# bi = BasicIFS(
#     vertices=SIMPLEX[4],
#     midpoint="f(A, B) = ((B - A) * 0.5) + B",
# )

bi = BasicIFS(
    vertices=SIMPLEX[4],
    midpoint="f(A, B) = (A + B) / 2",
)

In [8]:
points = bi.run(20000)

In [9]:
def rotate_z(x: float, y: float, z: float, theta: float) -> tuple[float, float, float]:
    """
    Takes current camera position and rotates on z-axis by theta.

    See: https://community.plotly.com/t/rotating-3d-plots-with-plotly/34776/4
    """
    w = x + 1j * y
    return np.real(np.exp(1j * theta) * w), np.imag(np.exp(1j * theta) * w), z


def rotate_x(x: float, y: float, z: float, theta: float) -> tuple[float, float, float]:
    v = y + 1j * z
    v_prime = np.exp(1j * theta) * v
    y_new = np.real(v_prime)
    z_new = np.imag(v_prime)
    return x, y_new, z_new


def rotate_y(x: float, y: float, z: float, theta: float) -> tuple[float, float, float]:
    v = z + 1j * x
    v_prime = np.exp(-1j * theta) * v
    x_new = np.imag(v_prime)
    z_new = np.real(v_prime)
    return x_new, y, z_new

In [10]:
x, y, z = 0, 2.5, 0
x, y, z = rotate_x(x, y, z, -0.35)
x, y, z = rotate_y(x, y, z, 0.18)
x, y, z = rotate_z(x, y, z, 0.85)

In [33]:
fig = v4d(
    points,
    width=600,
    height=500,
    camera=dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0.25, y=0, z=0.25),
        eye=dict(x=x, y=y, z=z),
    ),
)
fig

In [34]:
fig.write_html("../data/visualizations/sierpinski_4D_rotation.html")

In [28]:
class PointComputer:
    def __init__(self, function: str | Callable[[Point, Point], Point]) -> None:
        if isinstance(function, str):
            self.function = string_to_function(function)
        else:
            self.function = function

    def __call__(self, A: Point, B: Point) -> Point:
        return self.function(A, B)

In [29]:
pc = PointComputer("f(A, B) = (A + B)/2")

In [30]:
pc(np.array([0, 0]), np.array([1, 1]))

array([0.5, 0.5])